In [2]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
movies_data = pd.read_csv("D:\\GIT\\internship\\Day 26-28\\model_training\\Datasets\\movies.csv")

In [7]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [8]:
movies_data.shape

(4803, 24)

In [9]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [10]:
# selecting the relavent feature for recommandation

selected_features = ["genres","keywords","tagline","cast","director"]
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [11]:
movies_data.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [12]:
# replacing null value with null string
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna("")

In [13]:
movies_data.isnull().sum()

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64

In [14]:
# combining the all five selected features

combined_features = movies_data['genres'] + movies_data['keywords'] + movies_data['tagline'] + movies_data['cast'] + movies_data['director']
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object


In [15]:
# converting the text data into feactures vectors

vectorizer = TfidfVectorizer()


In [16]:
feature_vectores = vectorizer.fit_transform(combined_features)

In [17]:
print(feature_vectores)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 111820 stored elements and shape (4803, 27580)>
  Coords	Values
  (0, 245)	0.07858581760804864
  (0, 444)	0.09109355212252296
  (0, 8936)	0.11806131645084655
  (0, 21836)	0.09966592997173947
  (0, 9304)	0.27385789694599244
  (0, 4456)	0.2179967521551067
  (0, 10193)	0.1653243242081299
  (0, 22916)	0.3366875640692919
  (0, 26334)	0.130597230719163
  (0, 4676)	0.24529097704249617
  (0, 22778)	0.27385789694599244
  (0, 24197)	0.07518543993419267
  (0, 27161)	0.12619886579371511
  (0, 18164)	0.08690831799482268
  (0, 18558)	0.26121683351048536
  (0, 21521)	0.15623566659433683
  (0, 27182)	0.23480088356130557
  (0, 27540)	0.19771357974524179
  (0, 21487)	0.21550128478931552
  (0, 22464)	0.2020708956871175
  (0, 26547)	0.19638671147741735
  (0, 23267)	0.16128139780622522
  (0, 14550)	0.22368093636480682
  (0, 16614)	0.1569607387745327
  (0, 20729)	0.27385789694599244
  :	:
  (4801, 18790)	0.22450140828104795
  (4801, 11776)	0.3005

#Cosine Similarity


In [18]:
# gettting a similarity scores uing cosine similarity
similarity = cosine_similarity(feature_vectores)


In [19]:
print(similarity)

[[1.         0.06865296 0.01492221 ... 0.         0.         0.        ]
 [0.06865296 1.         0.02799128 ... 0.01243107 0.         0.        ]
 [0.01492221 0.02799128 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01243107 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [20]:
similarity.shape

(4803, 4803)

In [23]:
movie_name = input("Enter movie name: ")

In [24]:
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [25]:
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Gandhi', 'Bandits']


In [26]:
close_match = find_close_match[0]
print(close_match)

Gandhi


In [27]:
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

2024


In [28]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, np.float64(0.052199738264619544)), (1, np.float64(0.04082755586492458)), (2, np.float64(0.0)), (3, np.float64(0.004728169303087391)), (4, np.float64(0.029864957972767895)), (5, np.float64(0.0)), (6, np.float64(0.0)), (7, np.float64(0.0)), (8, np.float64(0.0)), (9, np.float64(0.0)), (10, np.float64(0.041456572074169286)), (11, np.float64(0.0)), (12, np.float64(0.0)), (13, np.float64(0.0052399880800645705)), (14, np.float64(0.036499062475999676)), (15, np.float64(0.019402021079320337)), (16, np.float64(0.0)), (17, np.float64(0.034965519832523904)), (18, np.float64(0.005262450829216118)), (19, np.float64(0.009487880926732473)), (20, np.float64(0.0)), (21, np.float64(0.0)), (22, np.float64(0.005175853448510204)), (23, np.float64(0.009333314187792915)), (24, np.float64(0.037752092431850764)), (25, np.float64(0.004403607090986419)), (26, np.float64(0.034923782873180084)), (27, np.float64(0.0)), (28, np.float64(0.0)), (29, np.float64(0.0)), (30, np.float64(0.0)), (31, np.float64(0.026979

In [29]:
len(similarity_score)

4803

In [30]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(2024, np.float64(1.0000000000000007)), (1787, np.float64(0.16785550902383328)), (1818, np.float64(0.12956397124483968)), (889, np.float64(0.11619138563985296)), (586, np.float64(0.11517663396283365)), (1043, np.float64(0.11447000959260942)), (1931, np.float64(0.11348260037084862)), (628, np.float64(0.11132220926436169)), (2986, np.float64(0.1113028865390286)), (3236, np.float64(0.11047018648856315)), (1299, np.float64(0.1069030254269157)), (3914, np.float64(0.1068787208468776)), (2547, np.float64(0.10678702520125322)), (617, np.float64(0.10594963141775346)), (1094, np.float64(0.10387622849576535)), (1510, np.float64(0.10377231401336692)), (4293, np.float64(0.10007038415800475)), (667, np.float64(0.09518588031910895)), (599, np.float64(0.09404821904184822)), (2233, np.float64(0.09326173910621913)), (2144, np.float64(0.09219271286753675)), (1775, np.float64(0.09147563919400178)), (4062, np.float64(0.09116559962721228)), (1527, np.float64(0.08881478659604815)), (4061, np.float64(0.08867

In [31]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Gandhi
2 . A Bridge Too Far
3 . Schindler's List
4 . The Thin Red Line
5 . The Monuments Men
6 . Miss Congeniality
7 . The Road
8 . Saving Private Ryan
9 . The Longest Day
10 . The Sound of Music
11 . Sweet Home Alabama
12 . Judgment at Nuremberg
13 . The Theory of Everything
14 . Agora
15 . Miracle at St. Anna
16 . 1941
17 . The Algerian
18 . Oliver Twist
19 . Hart's War
20 . Little Boy
21 . A Very Harold & Kumar Christmas
22 . The Statement
23 . The Best Years of Our Lives
24 . Mr. Holland's Opus
25 . Jack Brooks: Monster Slayer
26 . It's a Wonderful Afterlife
27 . Force 10 from Navarone
28 . The Believer
29 . American Desi


In [34]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Beloved
2 . Y Tu Mamá También
3 . Charlotte's Web
4 . The Color Purple
5 . Beastly
6 . The Ninth Gate
7 . The Chronicles of Riddick
8 . Mission: Impossible II
9 . The Crying Game
10 . Shade
11 . The Pursuit of Happyness
12 . 2012
13 . Blow Out
14 . An Unfinished Life
15 . Mooz-lum
16 . Wah-Wah
17 . Jefferson in Paris
18 . Erin Brockovich
19 . Run, Fatboy, Run
20 . For Colored Girls
21 . The Death and Life of Bobby Z
22 . Antz
23 . Casino Royale
24 . Wicked Blood
25 . Switchback
26 . Good Deeds
27 . Slow Burn
28 . Shooter
29 . W.
